In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import model
import torch
import numpy as np
import functools
import utils
from voc2012_dataset.dataset import VOC2012ClassSegmentation

torch.set_default_tensor_type('torch.DoubleTensor')

In [2]:
_input = np.random.rand(3, 512, 512)
_input = torch.from_numpy(_input).unsqueeze(0)
_input = torch.autograd.Variable(_input)

resnet = model.ResNet50()
output = model.FPN(*resnet.layers(), out_planes=256)(_input)
rpn = model.RPN(256, 512)

rpn_outputs = []
for o in output:
    rpn_outputs.append( rpn(o) )

In [3]:
# bbox_delta, class_logits, softmax, flatten rpn_outputs
bbox_delta, class_logits, softmax = zip(*rpn_outputs)

def merge(x, y):
    return torch.cat( (x, y), dim=1 )

bbox_delta = functools.reduce(merge, bbox_delta[1:], bbox_delta[0])
score = functools.reduce(merge, class_logits[1:], class_logits[0])
class_logits = functools.reduce(merge, class_logits[1:], class_logits[0])

### Get default anchors

Now we have bbox_delta and relative score from RPN, we should generate anchors and apply the RPN result to them.

**Important** make sure everything is generated from bottom to up in the FPN, and the size generated from RPN should be the same as anchors.

In [4]:
scales = [4, 8, 16, 32]
ratios = [0.5, 1, 2]
feature_strides =[4, 8, 16, 32]
input_image_shape = 512
anchors = utils. generate_pyramid_anchors(scales, ratios, input_image_shape, feature_strides)
# make sure the size is the same! and generate everything from bottom to up

### Generate RoIs.

Generate Regions of Interest, the process is: combine the output of `RPN` and anchors we got by `generate_pyramid_anchors` to get the final `fixed sized`, `refined` anchors. The anchors filter process involves the `nms`.

In [5]:
# output: [1, size_of_anchors, 4]
rp = model.RegionProposal()
rois = rp.forward(bbox_delta, score, anchors)
print(rois[:10])

Variable containing:
( 0  ,.,.) = 
  0.6224  0.0848  0.6390  0.0963
  0.5737  0.0579  0.5865  0.0657
  0.7892  0.1414  0.8045  0.1481
               ⋮                
  0.2803  0.2442  0.3134  0.2552
  0.1543  0.7838  0.1603  0.7955
  0.3307  0.1085  0.3420  0.1132
[torch.cuda.DoubleTensor of size 1x2000x4 (GPU 0)]



### Generate Target
Using output of RoI and ground truth bbox, class, mask to generate targets, which will used to compute the loss.

In [6]:
gen_targets = model.GenerateTarget()
voc_dataset = VOC2012ClassSegmentation('/home/louis/datasets/VOCdevkit/VOC2012')

image, bboxes, labels, bbox_masks = voc_dataset[5]

image = torch.from_numpy(image).double().cuda()
bboxes = torch.from_numpy(bboxes).double().cuda()
labels = torch.from_numpy(labels).double().cuda()
bbox_masks = torch.from_numpy(bbox_masks).double().cuda()

In [7]:
generate_rpn_targets = model.GenerateRPNTargets()
rpn_class, rpn_bounding_delta = generate_rpn_targets.forward(torch.from_numpy(anchors).cuda(), bboxes)
print(rpn_bounding_delta.size())

torch.Size([1000, 4])


In [8]:
rois, bbox, mask, classes = gen_targets.forward(rois, bboxes, bbox_masks, labels) 

In [9]:
print(rois.size())
print(bbox.size())
print(mask.size())
print(classes.size())

torch.Size([72, 4])
torch.Size([72, 4])
torch.Size([72, 28, 28])
torch.Size([72])


### Predict!!

Now we have all the rois and corresponding ground truth, it's time to do predictions.

We'll reuse the feature maps from FPN, predict bbox delta, corresponding class, and mask on each RoI, by using 2 different parallel networks.

In [10]:
cls_and_reg = model.ClsAndReg(21).cuda()

mrcnn_class_logits, mrcnn_probs, mrcnn_bbox = cls_and_reg(output, rois)

In [11]:
Mask = model.MaskPredict(21).cuda()
preds = Mask(output, rois)

### Now it's time to compute loss!

In [12]:
rpn_class_loss = model.compute_rpn_class_loss(rpn_class, class_logits.cuda())
bbox_delta = bbox_delta.cuda()

In [14]:
rpn_bbox_loss = model.compute_rpn_bbox_loss(rpn_bounding_delta, bbox_delta, rpn_class)

In [15]:
classes = classes.long()
mrcnn_class_loss = model.compute_mrcnn_class_loss(  classes, mrcnn_class_logits )

In [16]:
mrcnn_bbox_loss = model.compute_mrcnn_bbox_loss(bbox, classes, mrcnn_bbox)

In [18]:
mrcnn_mask_loss = model.compute_mrcnn_mask_loss(mask, classes, preds)

torch.Size([27, 28, 28])
torch.Size([27, 28, 28])
